# Module 10.1: Project Architecture & Best Practices - Interactive Analysis

## Overview

This notebook provides hands-on exploration of project architecture patterns and best practices for semiconductor ML projects. We'll walk through scaffolding projects, validating structures, and implementing configuration management patterns.

## Learning Objectives

- **Scaffold** semiconductor ML projects using standardized templates
- **Validate** project structures against industry best practices
- **Implement** configuration management and environment setup
- **Design** CLI interfaces with JSON outputs
- **Configure** structured logging for production environments

In [ ]:
# Standard imports
import json
import os
import sys
import tempfile
import subprocess
from pathlib import Path
from typing import Dict, Any, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml

# Set up paths relative to notebook location
NOTEBOOK_DIR = Path().resolve()
MODULE_DIR = NOTEBOOK_DIR.parent
PROJECT_ROOT = MODULE_DIR.parent.parent.parent
DATA_DIR = PROJECT_ROOT / "datasets"

# Add module to path for imports
sys.path.append(str(MODULE_DIR))

print(f"Notebook directory: {NOTEBOOK_DIR}")
print(f"Module directory: {MODULE_DIR}")
print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Data directory exists: {DATA_DIR.exists()}")

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

## 1. Project Scaffolding

Let's start by exploring the project scaffolding capabilities of our architecture pipeline.

In [ ]:
# Import our pipeline directly
from importlib.util import spec_from_file_location, module_from_spec

pipeline_path = MODULE_DIR / "10.1-project-architecture-pipeline.py"
spec = spec_from_file_location("project_architecture", pipeline_path)
project_arch = module_from_spec(spec)
spec.loader.exec_module(project_arch)

# Create pipeline instance
pipeline = project_arch.ProjectArchitecturePipeline()

print("✅ Project architecture pipeline loaded successfully")
print(f"Available project types: {list(project_arch.SEMICONDUCTOR_PROJECT_TYPES.keys())}")

### Explore Available Project Types

Our scaffolding system supports different types of semiconductor ML projects:

In [ ]:
# Display project types and their characteristics
project_types = project_arch.SEMICONDUCTOR_PROJECT_TYPES

for project_type, info in project_types.items():
    print(f"\n🔬 **{project_type.title()}**")
    print(f"   Description: {info['description']}")
    print(f"   Key Metrics: {', '.join(info['key_metrics'])}")
    print(f"   Sample Datasets: {', '.join(info['sample_datasets'])}")

### Scaffold a Classification Project

Let's create a semiconductor defect classification project using our scaffolding system:

In [ ]:
# Create a temporary directory for our demo project
demo_projects_dir = Path(tempfile.mkdtemp(prefix="semiconductor_projects_"))
print(f"Demo projects directory: {demo_projects_dir}")

# Scaffold a classification project
project_name = "wafer_defect_classifier"
project_type = "classification"

try:
    result = pipeline.scaffold(
        name=project_name,
        project_type=project_type,
        output_dir=demo_projects_dir,
        include_notebooks=True,
        include_docker=True
    )
    
    print("✅ Project scaffolded successfully!")
    print(f"Project path: {result['project_path']}")
    print(f"Project type: {result['project_type']}")
    
except Exception as e:
    print(f"❌ Error scaffolding project: {e}")

### Explore Generated Project Structure

Let's examine what was created:

In [ ]:
project_path = demo_projects_dir / project_name

def display_tree(directory, prefix="", max_depth=3, current_depth=0):
    """Display directory tree structure."""
    if current_depth >= max_depth:
        return
        
    items = sorted(directory.iterdir())
    dirs = [item for item in items if item.is_dir()]
    files = [item for item in items if item.is_file()]
    
    # Display directories first
    for i, item in enumerate(dirs):
        is_last_dir = (i == len(dirs) - 1 and len(files) == 0)
        current_prefix = "└── " if is_last_dir else "├── "
        print(f"{prefix}{current_prefix}{item.name}/")
        
        next_prefix = prefix + ("    " if is_last_dir else "│   ")
        display_tree(item, next_prefix, max_depth, current_depth + 1)
    
    # Display files
    for i, item in enumerate(files):
        is_last = (i == len(files) - 1)
        current_prefix = "└── " if is_last else "├── "
        print(f"{prefix}{current_prefix}{item.name}")

print(f"📁 Project Structure: {project_name}")
display_tree(project_path)

### Examine Generated Files

Let's look at some key files that were generated:

In [ ]:
# README.md
readme_path = project_path / "README.md"
if readme_path.exists():
    print("📄 README.md (first 20 lines):")
    with open(readme_path) as f:
        lines = f.readlines()[:20]
        for i, line in enumerate(lines, 1):
            print(f"{i:2d}: {line.rstrip()}")
    print("...")

In [ ]:
# Configuration file
config_path = project_path / "configs" / "config.yaml"
if config_path.exists():
    print("⚙️ Configuration (config.yaml):")
    with open(config_path) as f:
        config = yaml.safe_load(f)
        print(yaml.dump(config, default_flow_style=False, indent=2))

In [ ]:
# Environment template
env_template_path = project_path / ".env.template"
if env_template_path.exists():
    print("🔒 Environment Template (.env.template):")
    with open(env_template_path) as f:
        lines = f.readlines()[:15]  # Show first 15 lines
        for i, line in enumerate(lines, 1):
            print(f"{i:2d}: {line.rstrip()}")
    print("...")

## 2. Project Validation

Now let's validate our scaffolded project against best practices:

In [ ]:
# Validate the generated project
validation_result = pipeline.validate(project_path)

print(f"📊 Validation Results for {project_name}")
print(f"Valid: {'✅' if validation_result.is_valid else '❌'} {validation_result.is_valid}")
print(f"Score: {validation_result.score:.1f}/100.0")

if validation_result.errors:
    print(f"\n❌ Errors ({len(validation_result.errors)}):")
    for error in validation_result.errors:
        print(f"   • {error}")

if validation_result.warnings:
    print(f"\n⚠️ Warnings ({len(validation_result.warnings)}):")
    for warning in validation_result.warnings:
        print(f"   • {warning}")

if validation_result.suggestions:
    print(f"\n💡 Suggestions ({len(validation_result.suggestions)}):")
    for suggestion in validation_result.suggestions:
        print(f"   • {suggestion}")

print(f"\n📈 Score Breakdown:")
for metric, score in validation_result.metrics.items():
    print(f"   {metric}: {score:.1f} points")

### Visualize Validation Metrics

Let's create a visualization of the validation scores:

In [ ]:
# Create a bar chart of validation metrics
metrics = validation_result.metrics
metric_names = list(metrics.keys())
metric_scores = list(metrics.values())

plt.figure(figsize=(10, 6))
bars = plt.bar(metric_names, metric_scores, color='skyblue', alpha=0.7, edgecolor='navy')

# Add value labels on bars
for bar, score in zip(bars, metric_scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{score:.1f}', ha='center', va='bottom', fontweight='bold')

plt.title(f'Project Validation Scores: {project_name}', fontsize=16, fontweight='bold')
plt.ylabel('Score', fontsize=12)
plt.xlabel('Validation Metric', fontsize=12)
plt.xticks(rotation=45)
plt.ylim(0, max(metric_scores) * 1.1)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Overall Score: {validation_result.score:.1f}/100.0")
print(f"Status: {'✅ PASS' if validation_result.is_valid else '❌ FAIL'}")

## 3. Detailed Structure Linting

Let's perform detailed linting to check for common anti-patterns and issues:

In [ ]:
# Perform detailed linting
lint_result = pipeline.lint_structure(project_path)

print("🔍 Detailed Linting Results")
print("=" * 40)

# Display validation summary
validation = lint_result['validation']
print(f"Validation Status: {'✅ PASS' if validation['is_valid'] else '❌ FAIL'}")
print(f"Overall Score: {validation['score']:.1f}/100.0")

# Display detailed checks
detailed = lint_result['detailed_checks']

print("\n🚨 Anti-patterns Check:")
anti_patterns = detailed.get('anti_patterns', [])
if anti_patterns:
    for pattern in anti_patterns:
        print(f"   ❌ {pattern}")
else:
    print("   ✅ No anti-patterns detected")

print("\n📝 Naming Issues:")
naming_issues = detailed.get('naming_issues', [])
if naming_issues:
    for issue in naming_issues:
        print(f"   ⚠️ {issue}")
else:
    print("   ✅ No naming issues detected")

print("\n📦 Import Issues:")
import_issues = detailed.get('import_issues', [])
if import_issues:
    for issue in import_issues:
        print(f"   ⚠️ {issue}")
else:
    print("   ✅ No import issues detected")

## 4. CLI Interface Testing

Let's test the CLI interface of our pipeline using subprocess calls:

In [ ]:
# Test CLI help command
pipeline_script = MODULE_DIR / "10.1-project-architecture-pipeline.py"

print("🖥️ Testing CLI Interface")
print("=" * 30)

# Test help command
try:
    result = subprocess.run([
        'python', str(pipeline_script), '--help'
    ], capture_output=True, text=True, timeout=10)
    
    print("📋 CLI Help Output:")
    print(result.stdout)
    
except subprocess.TimeoutExpired:
    print("❌ CLI help command timed out")
except Exception as e:
    print(f"❌ Error running CLI help: {e}")

In [ ]:
# Test scaffold command via CLI
cli_project_name = "cli_test_project"
cli_output_dir = demo_projects_dir

try:
    result = subprocess.run([
        'python', str(pipeline_script), 'scaffold',
        '--name', cli_project_name,
        '--type', 'regression',
        '--output', str(cli_output_dir),
        '--no-docker'  # Skip Docker for faster testing
    ], capture_output=True, text=True, timeout=30)
    
    if result.returncode == 0:
        print("✅ CLI scaffold command successful!")
        
        # Parse JSON output
        try:
            output_data = json.loads(result.stdout)
            print(f"📁 Created project: {output_data['project_path']}")
            print(f"📊 Project type: {output_data['project_type']}")
            print(f"✨ Features: {len(output_data['metadata']['compliance_features'])} compliance features")
        except json.JSONDecodeError:
            print("⚠️ Could not parse JSON output")
            print("Raw output:", result.stdout[:200], "...")
    else:
        print(f"❌ CLI scaffold command failed with exit code {result.returncode}")
        print("Error output:", result.stderr)
        
except subprocess.TimeoutExpired:
    print("❌ CLI scaffold command timed out")
except Exception as e:
    print(f"❌ Error running CLI scaffold: {e}")

In [ ]:
# Test validation command via CLI
if (cli_output_dir / cli_project_name).exists():
    try:
        result = subprocess.run([
            'python', str(pipeline_script), 'validate',
            '--project-path', str(cli_output_dir / cli_project_name)
        ], capture_output=True, text=True, timeout=15)
        
        if result.returncode == 0:
            print("✅ CLI validation command successful!")
            
            # Parse JSON output
            try:
                output_data = json.loads(result.stdout)
                validation_data = output_data['validation']
                
                print(f"📊 Validation Score: {validation_data['score']:.1f}/100.0")
                print(f"✅ Valid: {validation_data['is_valid']}")
                print(f"❌ Errors: {len(validation_data['errors'])}")
                print(f"⚠️ Warnings: {len(validation_data['warnings'])}")
                
            except json.JSONDecodeError:
                print("⚠️ Could not parse JSON output")
                print("Raw output:", result.stdout[:200], "...")
        else:
            print(f"❌ CLI validation command failed with exit code {result.returncode}")
            print("Error output:", result.stderr)
            
    except subprocess.TimeoutExpired:
        print("❌ CLI validation command timed out")
    except Exception as e:
        print(f"❌ Error running CLI validation: {e}")
else:
    print("⚠️ CLI test project not found, skipping validation test")

## 5. Configuration Management Patterns

Let's explore the configuration management patterns in our generated projects:

In [ ]:
# Demonstrate configuration loading pattern
import os
from dataclasses import dataclass
from typing import Optional

@dataclass
class DemoConfig:
    """Example configuration class for semiconductor ML projects."""
    project_name: str
    environment: str
    log_level: str
    random_seed: int
    tolerance: float
    spec_low: float
    spec_high: float
    
    @classmethod
    def from_env(cls, config_file: Optional[Path] = None):
        """Load configuration from environment and YAML file."""
        # Simulate environment variables
        env_vars = {
            'PROJECT_NAME': 'demo_semiconductor_project',
            'ENVIRONMENT': 'development',
            'LOG_LEVEL': 'INFO',
            'RANDOM_SEED': '42',
            'TOLERANCE': '2.0',
            'SPEC_LOW': '60.0',
            'SPEC_HIGH': '100.0'
        }
        
        return cls(
            project_name=env_vars.get('PROJECT_NAME', 'unnamed_project'),
            environment=env_vars.get('ENVIRONMENT', 'development'),
            log_level=env_vars.get('LOG_LEVEL', 'INFO'),
            random_seed=int(env_vars.get('RANDOM_SEED', '42')),
            tolerance=float(env_vars.get('TOLERANCE', '2.0')),
            spec_low=float(env_vars.get('SPEC_LOW', '60.0')),
            spec_high=float(env_vars.get('SPEC_HIGH', '100.0'))
        )

# Load configuration
config = DemoConfig.from_env()

print("⚙️ Configuration Loading Example")
print(f"Project Name: {config.project_name}")
print(f"Environment: {config.environment}")
print(f"Log Level: {config.log_level}")
print(f"Random Seed: {config.random_seed}")
print(f"Manufacturing Tolerance: {config.tolerance}")
print(f"Spec Limits: {config.spec_low} - {config.spec_high}")

## 6. Manufacturing Metrics Implementation

Let's implement and test semiconductor-specific metrics that should be included in projects:

In [ ]:
def compute_semiconductor_metrics(y_true, y_pred, tolerance=2.0, 
                                spec_low=60.0, spec_high=100.0,
                                cost_per_unit=1.0):
    """Compute semiconductor manufacturing metrics."""
    
    # Prediction Within Spec (PWS)
    pws = np.mean((y_pred >= spec_low) & (y_pred <= spec_high))
    
    # Estimated Loss from prediction errors
    loss_components = np.maximum(0, np.abs(y_true - y_pred) - tolerance)
    estimated_loss = float(np.sum(loss_components) * cost_per_unit)
    
    # Yield Rate (assuming higher values = better yield)
    yield_rate = np.mean(y_pred >= spec_low)
    
    # Process Capability Index (simplified)
    if len(y_pred) > 1:
        process_std = np.std(y_pred)
        spec_range = spec_high - spec_low
        cp_index = spec_range / (6 * process_std) if process_std > 0 else float('inf')
    else:
        cp_index = 0.0
    
    return {
        'PWS': pws,
        'Estimated_Loss': estimated_loss,
        'Yield_Rate': yield_rate,
        'Cp_Index': cp_index
    }

# Generate synthetic semiconductor data for demonstration
np.random.seed(RANDOM_SEED)
n_samples = 1000

# Simulate process parameters
temperature = np.random.normal(450, 15, n_samples)
pressure = np.random.normal(2.5, 0.3, n_samples)
flow_rate = np.random.normal(120, 10, n_samples)

# Simulate yield (target variable)
noise = np.random.normal(0, 3, n_samples)
y_true = (0.2 * temperature + 10 * pressure + 0.1 * flow_rate + noise)

# Simulate predictions (with some error)
prediction_noise = np.random.normal(0, 2, n_samples)
y_pred = y_true + prediction_noise

# Compute metrics
metrics = compute_semiconductor_metrics(
    y_true, y_pred, 
    tolerance=config.tolerance,
    spec_low=config.spec_low,
    spec_high=config.spec_high
)

print("🏭 Semiconductor Manufacturing Metrics")
print("=" * 40)
for metric_name, value in metrics.items():
    if metric_name == 'Estimated_Loss':
        print(f"{metric_name}: ${value:.2f}")
    elif metric_name in ['PWS', 'Yield_Rate']:
        print(f"{metric_name}: {value:.1%}")
    else:
        print(f"{metric_name}: {value:.3f}")

### Visualize Manufacturing Metrics

Let's create visualizations to understand these semiconductor-specific metrics:

In [ ]:
# Create a comprehensive visualization of manufacturing metrics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Semiconductor Manufacturing Metrics Dashboard', fontsize=16, fontweight='bold')

# Plot 1: Prediction vs True values with spec limits
ax1 = axes[0, 0]
ax1.scatter(y_true, y_pred, alpha=0.6, s=20)
ax1.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', alpha=0.8, label='Perfect Prediction')
ax1.axhline(y=config.spec_low, color='orange', linestyle='--', alpha=0.7, label='Spec Limits')
ax1.axhline(y=config.spec_high, color='orange', linestyle='--', alpha=0.7)
ax1.axvline(x=config.spec_low, color='orange', linestyle='--', alpha=0.7)
ax1.axvline(x=config.spec_high, color='orange', linestyle='--', alpha=0.7)
ax1.set_xlabel('True Values')
ax1.set_ylabel('Predicted Values')
ax1.set_title('Predictions vs True Values')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Distribution of predictions with spec limits
ax2 = axes[0, 1]
ax2.hist(y_pred, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
ax2.axvline(x=config.spec_low, color='red', linestyle='--', linewidth=2, label=f'Lower Spec ({config.spec_low})')
ax2.axvline(x=config.spec_high, color='red', linestyle='--', linewidth=2, label=f'Upper Spec ({config.spec_high})')
ax2.axvline(x=np.mean(y_pred), color='green', linestyle='-', linewidth=2, label=f'Mean ({np.mean(y_pred):.1f})')
ax2.set_xlabel('Predicted Values')
ax2.set_ylabel('Frequency')
ax2.set_title('Distribution of Predictions')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Error distribution with tolerance
ax3 = axes[1, 0]
errors = np.abs(y_true - y_pred)
ax3.hist(errors, bins=50, alpha=0.7, color='lightcoral', edgecolor='black')
ax3.axvline(x=config.tolerance, color='red', linestyle='--', linewidth=2, label=f'Tolerance ({config.tolerance})')
ax3.axvline(x=np.mean(errors), color='blue', linestyle='-', linewidth=2, label=f'Mean Error ({np.mean(errors):.2f})')
ax3.set_xlabel('Absolute Error')
ax3.set_ylabel('Frequency')
ax3.set_title('Prediction Error Distribution')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Metrics summary
ax4 = axes[1, 1]
metric_names = list(metrics.keys())
metric_values = [metrics[name] for name in metric_names]

# Normalize metrics for display (0-1 scale)
normalized_values = []
for name, value in zip(metric_names, metric_values):
    if name in ['PWS', 'Yield_Rate']:
        normalized_values.append(value)  # Already 0-1
    elif name == 'Cp_Index':
        normalized_values.append(min(value / 2.0, 1.0))  # Cap at 2.0 for display
    else:  # Estimated_Loss
        normalized_values.append(max(0, 1 - value / 1000))  # Invert and scale

bars = ax4.bar(range(len(metric_names)), normalized_values, 
               color=['green', 'red', 'blue', 'purple'], alpha=0.7)
ax4.set_xticks(range(len(metric_names)))
ax4.set_xticklabels(metric_names, rotation=45)
ax4.set_ylabel('Normalized Score (0-1)')
ax4.set_title('Manufacturing Metrics Summary')
ax4.set_ylim(0, 1)

# Add value labels on bars
for bar, original_value, name in zip(bars, metric_values, metric_names):
    if name == 'Estimated_Loss':
        label = f'${original_value:.0f}'
    elif name in ['PWS', 'Yield_Rate']:
        label = f'{original_value:.1%}'
    else:
        label = f'{original_value:.2f}'
    
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             label, ha='center', va='bottom', fontsize=9, fontweight='bold')

ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\n📊 Summary Statistics:")
print(f"   Samples: {n_samples:,}")
print(f"   Within Spec: {int(metrics['PWS'] * n_samples):,} ({metrics['PWS']:.1%})")
print(f"   Mean Absolute Error: {np.mean(errors):.2f}")
print(f"   Samples Within Tolerance: {int(np.mean(errors <= config.tolerance) * n_samples):,} ({np.mean(errors <= config.tolerance):.1%})")

## 7. Testing Different Project Types

Let's test scaffolding different types of semiconductor projects to see how the templates adapt:

In [ ]:
# Test all project types
project_type_results = {}

for proj_type in project_types.keys():
    test_name = f"test_{proj_type}_project"
    
    try:
        result = pipeline.scaffold(
            name=test_name,
            project_type=proj_type,
            output_dir=demo_projects_dir,
            include_notebooks=False,  # Skip notebooks for faster testing
            include_docker=False      # Skip Docker for faster testing
        )
        
        # Validate the created project
        validation = pipeline.validate(Path(result['project_path']))
        
        project_type_results[proj_type] = {
            'success': True,
            'path': result['project_path'],
            'score': validation.score,
            'valid': validation.is_valid,
            'features': len(result['metadata']['compliance_features'])
        }
        
        print(f"✅ {proj_type.title()}: Created and validated (Score: {validation.score:.1f})")
        
    except Exception as e:
        project_type_results[proj_type] = {
            'success': False,
            'error': str(e)
        }
        print(f"❌ {proj_type.title()}: Failed - {e}")

print(f"\n📋 Project Type Testing Summary:")
successful = sum(1 for r in project_type_results.values() if r.get('success', False))
print(f"   Successful: {successful}/{len(project_types)}")
print(f"   Average Score: {np.mean([r['score'] for r in project_type_results.values() if r.get('success', False)]):.1f}")

### Compare Project Types

Let's visualize the differences between project types:

In [ ]:
# Create comparison visualization
successful_results = {k: v for k, v in project_type_results.items() if v.get('success', False)}

if successful_results:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Validation scores comparison
    types = list(successful_results.keys())
    scores = [successful_results[t]['score'] for t in types]
    
    bars1 = ax1.bar(types, scores, color='lightblue', alpha=0.7, edgecolor='navy')
    ax1.set_title('Validation Scores by Project Type', fontweight='bold')
    ax1.set_ylabel('Validation Score')
    ax1.set_ylim(0, 100)
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, score in zip(bars1, scores):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                f'{score:.1f}', ha='center', va='bottom', fontweight='bold')
    
    # Features comparison
    features = [successful_results[t]['features'] for t in types]
    
    bars2 = ax2.bar(types, features, color='lightgreen', alpha=0.7, edgecolor='darkgreen')
    ax2.set_title('Compliance Features by Project Type', fontweight='bold')
    ax2.set_ylabel('Number of Features')
    ax2.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, feature_count in zip(bars2, features):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                f'{feature_count}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Project Type Analysis:")
    for proj_type, result in successful_results.items():
        status = "✅ VALID" if result['valid'] else "⚠️ ISSUES"
        print(f"   {proj_type.title()}: {status} (Score: {result['score']:.1f}, Features: {result['features']})")
else:
    print("⚠️ No successful project types to compare")

## 8. Cleanup and Summary

Let's clean up our temporary projects and summarize what we've learned:

In [ ]:
# Cleanup temporary projects
import shutil

try:
    if demo_projects_dir.exists():
        shutil.rmtree(demo_projects_dir)
        print(f"🧹 Cleaned up temporary projects directory: {demo_projects_dir}")
except Exception as e:
    print(f"⚠️ Could not clean up temporary directory: {e}")

print("\n🎯 Module 10.1 Summary")
print("=" * 50)
print("\n✅ **What We Accomplished:**")
print("   • Created standardized project scaffolding system")
print("   • Implemented comprehensive project validation")
print("   • Demonstrated CLI interface with JSON outputs")
print("   • Explored configuration management patterns")
print("   • Implemented semiconductor-specific metrics")
print("   • Tested different project types and templates")

print("\n🔑 **Key Concepts Covered:**")
print("   • Standardized directory structures for ML projects")
print("   • Multi-tier configuration management (env vars, YAML, constants)")
print("   • Data versioning and path resolution strategies")
print("   • CLI consistency with train/evaluate/predict patterns")
print("   • Structured logging with JSON outputs")
print("   • Secrets management with environment variables")
print("   • Manufacturing-specific metrics (PWS, Estimated Loss, etc.)")

print("\n🏭 **Semiconductor Manufacturing Focus:**")
print("   • Process parameter templates and validation")
print("   • Yield prediction and defect classification structures")
print("   • Specification compliance and tolerance metrics")
print("   • Equipment monitoring and time series patterns")

print("\n🚀 **Next Steps:**")
print("   • Apply these patterns to your own semiconductor ML projects")
print("   • Customize templates for your specific manufacturing processes")
print("   • Integrate with your existing CI/CD and deployment pipelines")
print("   • Extend metrics and validation rules for your domain")

print("\n📚 **Resources for Further Learning:**")
print("   • 10.1-project-architecture-fundamentals.md - Detailed theory")
print("   • 10.1-project-architecture-quick-ref.md - Quick reference guide")
print("   • 10.1-project-architecture-pipeline.py - Full implementation")
print("   • projects/starter/template/ - Reusable project template")